In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat

data pre-proccessing

In [ ]:
file = loadmat('/content/drive/MyDrive/spike_data.mat')
data = file['wf']
data_concat = data[0][0]
for i in range(0, 96):
  for j in range(0, 5):
    if not(i==0 and j==0):
      data_concat = np.append(data_concat, data[i][j], axis=0)

In [ ]:
def pca(data, dim=2):
    m = tf.cast(data.shape[0], dtype='float32')                             # number of data
    data_new = data - tf.reduce_mean(data, axis=0, keepdims=True)           # 1.centralize data
    # data_new = data_new/tf.math.reduce_variance(data, axis=1, keepdims=True)
    cov_matrix = tf.matmul(data_new, data_new, transpose_a=True)/(m - 1)    # 2.compute cov matrix
    e, v = tf.linalg.eigh(cov_matrix)                                       # 3.eigen decomposition
    e_indices = tf.math.top_k(e, k=dim)[1]                                  # 4-1.pick indices of the max 2 eigenvalues
    v_new = tf.gather(v, indices=e_indices)                                 # 4-2.choose corresponding eigenvectors
    data_dim_reduced = tf.matmul(v_new, data_new, transpose_b=True)         # 5.reduce dimension
    return data_dim_reduced

plot all data points with dimensionality reduced:

In [ ]:
fig = plt.figure(figsize=(15,15))
ax1 = fig.add_subplot(111)
ax1.set_title('reduce dimension by PCA')
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim((-1200,1200))
plt.ylim((-300,1800))
x_ticks = np.arange(-1200, 1200, 100)
y_ticks = np.arange(-300, 1800, 50)
plt.xticks(x_ticks)
plt.yticks(y_ticks)

data_dim_reduced = pca(data_concat)
x = data_dim_reduced[0,:]
y = data_dim_reduced[1,:]
ax1.scatter(x, y, c = 'b', marker = '.')
ax1.plot(x, 3*x)

plt.show()

plot three clusters:

In [ ]:
data_dim_reduced = np.array(data_dim_reduced)

a, b, c = [], [], []
orig_a, orig_b, orig_c = [], [], []
for i in np.where(data_dim_reduced[1,:] > 200)[0]:
  x_value = data_dim_reduced[0][i]
  y_value = data_dim_reduced[1][i]
  above_line = 3 * x_value
  if x_value < -100:
    a.append([x_value, y_value])
    orig_a.append(data_concat[:,i])
  elif x_value > -100 and y_value > above_line:
    b.append([x_value, y_value])
    orig_b.append(data_concat[:,i])
  elif y_value < above_line:
    c.append([x_value, y_value])
    orig_c.append(data_concat[:,i])

cluster_a = np.array(a)
cluster_b = np.array(b)
cluster_c = np.array(c)

cluster_orig_a = np.array(orig_a)
cluster_orig_b = np.array(orig_b)
cluster_orig_c = np.array(orig_c)

In [ ]:
fig = plt.figure(figsize=(5,5))
ax1 = fig.add_subplot(111)
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim((-1200,1200))
plt.ylim((-300,1800))

x_a = cluster_a[:,0]
y_a = cluster_a[:,1]

x_b = cluster_b[:,0]
y_b = cluster_b[:,1]

x_c = cluster_c[:,0]
y_c = cluster_c[:,1]

ax1.scatter(x_a, y_a, c = 'r', marker = '.')
ax1.scatter(x_b, y_b, c = 'b', marker = '.')
ax1.scatter(x_c, y_c, c = 'g', marker = '.')

plt.show()

fig2 = plt.figure(figsize=(12,12))
ax1 = fig2.add_subplot(311)
ax2 = fig2.add_subplot(312)
ax3 = fig2.add_subplot(313)
plt.xlabel('t')
plt.ylabel('haha')
plt.xlim((0,64))
plt.ylim((-200,200))

x = np.arange(64)

# cluster a
for i in range(cluster_orig_a.shape[0]):
  ax1.plot(x, cluster_orig_a[i], c = 'r', marker = '.')

# cluster b
for i in range(cluster_orig_b.shape[0]):
  ax2.plot(x, cluster_orig_b[i], c = 'b', marker = '.')

# cluster c
for i in range(cluster_orig_c.shape[0]):
  ax3.plot(x, cluster_orig_c[i], c = 'g', marker = '.')

plt.show()

In [ ]:
fig3 = plt.figure(figsize=(30,10))
ax1 = fig3.add_subplot(111)
plt.xlabel('t')
plt.ylabel('haha')
plt.xlim((0,64))
plt.ylim((-200,200))

x = np.arange(64)

for i in range(1900):
  ax1.plot(x, data_concat[:,i], c = 'r', marker = '.')

plt.show()